# Sumplete solver using Genetic Algorithm

In [2]:
import random
import math

In [3]:
class Sumplete:
    def __init__(self, k: int):
        MAX_NUMBER = 9
        MIN_K = 3
        if k < MIN_K:
            k = MIN_K
        
        self.grid = []
        for i in range(k):
            row = [random.randint(1,MAX_NUMBER) for i in range(k)]
            self.grid.append(row)
            
        self.row_totals = []
        self.col_totals = []
        # selecting a random set of choices of the rows and cols to be the goal
        self.sol_grid = []
        for row in self.grid:
            sol_row = [row[j] for j in range(k)]
            for i in range(k):
                # delete about a 1/3 of elements from a row to get the solution
                if random.randint(1,3) == 1:
                    sol_row[i] = -1
            self.sol_grid.append(sol_row)
            self.row_totals.append(sum(list(filter(lambda x: x!=-1, sol_row))))
        
        # There is definatetly a more efficient way to do this, but efficiency not really a big problem soooooo....
        for i in range(k):
            sol_col = []
            for j in range(k):
                sol_col.append(self.sol_grid[j][i])
                
            self.col_totals.append(sum(list(filter(lambda x: x!=-1, sol_col))))
  
    def print_grid_and_totals(self):
        for r in self.grid:
            print(r)
            
        print('\n')
            
        print(self.row_totals)
        print(self.col_totals)
        
    def print_sol_grid(self):
        for r in self.sol_grid:
            print(r)
            
    def get_grid_and_totals(self):
        temp = [r for r in self.grid]
        temp.append(self.row_totals)
        temp.append(self.col_totals)
        return temp
            

Sumplete(7).print_grid_and_totals()
            
            

[9, 7, 6, 7, 1, 5, 8]
[5, 2, 1, 2, 3, 9, 9]
[9, 2, 9, 9, 2, 6, 4]
[1, 5, 5, 4, 6, 8, 7]
[6, 5, 8, 8, 1, 7, 9]
[9, 1, 8, 3, 5, 4, 4]
[1, 2, 2, 3, 7, 3, 5]


[27, 23, 15, 16, 29, 18, 8]
[21, 13, 8, 34, 13, 17, 30]


In [4]:
# encoding = could flatten the matrix into an array of k^2 elements?? and the encoding could be a 1 for include cell and 0 for delete cell?
# fitness function = sum of (square?) differences between actual rows and cols and target rows ans cols, normalised 
# selection??
# params:
# POP_SIZE
# MUTATION_RATE
# CROSSOVER_RATE??
# GENERATIONS
# BREEDING_RATE

class GA_solver:
    def __init__(self, problem, pop_size, mutation_rate, num_generations, binary_fitness):
        # parse problem
        self.problem_k = len(problem[0])
        self.problem_grid = problem[0:self.problem_k]
        self.problem_row_totals = problem[self.problem_k]
        self.problem_col_totals = problem[self.problem_k+1]
        
        self.using_binary_fitness = binary_fitness
        
        self.solution_foiund_in_gen = num_generations
        
        print("solving problem grid:")
        print(self.problem_grid)
        
        # initialise population
        self.population = []
        for _ in range(pop_size):
            self.population.append(Individual(self.problem_grid, self.problem_k, self.problem_row_totals, self.problem_col_totals, self.using_binary_fitness))
            
        self.solve(num_generations, pop_size, mutation_rate)
    
    # this function only works if population is sorted. used for getting bests at the end of a run.
    def get_best(self):
        return self.population[0].fitness
    
    # TODO: seperate crossover and mutation into 2 funcs, so I have control over crossover_rate and mutation_rate, rather than just mutation rate.
    def breed(self, individual1, individual2, mutation_chance):
        # using code inspired by https://www.geeksforgeeks.org/genetic-algorithms/
        chrome1 = individual1.get_chromosome()
        chrome2 = individual2.get_chromosome()
        new_chrome = []
        for g1, g2 in zip(chrome1, chrome2):
            p = random.random()
            crossover_chance = (1 - mutation_chance) / 2
            
            if p < mutation_chance:
                # muatate!
                new_chrome.append(random.randint(0,1))
            elif p < mutation_chance + crossover_chance:
                # 45% chance to pick gene from individual 1
                new_chrome.append(g1)
            else:
                # 45% chance to pick gene from individual 2
                new_chrome.append(g2)
                
        child = Individual(self.problem_grid, self.problem_k, self.problem_row_totals, self.problem_col_totals, self.using_binary_fitness)
        child.set_chromosome(new_chrome)
        return child
        
    # ranked selection using elitism
    # possible todo: implement different selection: roulette, tournement
    def selection(self, pop_size, mutation_rate):
        # elitism - 10% of best of pop goes into next population
        # the rest is generated from the top 50% of the old population
        new_pop = []
        
        pop = self.population
        
        # get elite 10%
        num_of_elites = pop_size // 20
        elites = pop[:num_of_elites+1]
        new_pop.extend(elites)
        
        # breed the rest of the new population
        top_50 = pop[:(pop_size // 2)+1]
        for _ in range(num_of_elites, pop_size+1):
            # select 2 random individuals from top 50% to breed
            # possible todo: make it so parents cannot be the same?
            parent1 = random.choice(top_50)
            parent2 = random.choice(top_50)
            child = self.breed(parent1, parent2, mutation_rate)
            new_pop.append(child)
        
        # setting new population here, could also return instead
        self.population = new_pop
        
    
    def solve(self, generations, pop_size, mutation_rate):
        for gen in range(generations):
            self.population = sorted(self.population, key=lambda x: x.calc_fitness(self.problem_k, self.problem_row_totals, self.problem_col_totals))
            # sort population, find the current best fitness, if it zero then terminate the loop as we have a solution
            if self.population[0].fitness == 0:
                # solution found
                print("SOLUTION FOUND!")
                print("In gen:", gen)
                self.solution_foiund_in_gen = gen
                self.population[0].pretty_print_chromosome()
                print(self.population[0].calc_fitness(self.problem_k, self.problem_row_totals, self.problem_col_totals))
                break
            else:
                print("Gen {}: Best individual fitness = {}".format(gen, self.population[0].fitness))
                #self.population[0].pretty_print_chromosome()
            # if no solution is found, breed a new population
            self.selection(pop_size, mutation_rate)
    
    
# problem size == k
class Individual:
    def __init__(self, problem_grid, problem_size, problem_row_totals, problem_col_totals, use_bin_fit):
        # generate initial solution
        self.chromosome = [random.randint(0,1) for _ in range(problem_size**2)]
        
        # todo:
        # awful programming here, need to pick one or the other, global or local vars
        # flatten problem grid to match chromosome encoding
        self.problem = [item for sublist in problem_grid for item in sublist]
        self.problem_rows = problem_row_totals
        self.problem_cols = problem_col_totals
        self.prob_size = problem_size
        
        self.use_binary_fit = use_bin_fit
        
        self.calc_fitness(problem_size, problem_row_totals, problem_col_totals)


    def get_chromosome(self):
        return self.chromosome

    def set_chromosome(self, new_chrome):
        self.chromosome = new_chrome
        # recalculating fitness although I may remove this and move it elsewhere...
        self.calc_fitness(self.prob_size, self.problem_rows, self.problem_cols)
      
    # fitness equals sum of the square differences between the chromosome's row/col totals and target row/col totals    
    def scaling_fitness(self, k, problem_row_sums, problem_col_sums):
        sum_of_square_differences = 0
        curr_row_sums = self.calc_rows(k)
        curr_col_sums = self.calc_cols(k)
        for i in range(k):
            sum_of_square_differences += (curr_col_sums[i] - problem_col_sums[i]) ** 2
            sum_of_square_differences += (curr_row_sums[i] - problem_row_sums[i]) ** 2
        self.fitness = 1 - (1 / (1 + sum_of_square_differences))
        return self.fitness
    
    def calc_fitness(self, k, problem_row_sums, problem_col_sums):
        if self.use_binary_fit:
            return self.binary_fitness(k, problem_row_sums, problem_col_sums)
        else:
            return self.scaling_fitness(k, problem_row_sums, problem_col_sums)
    
    # defined in specification as a fitness function that only returns 0  (not a correct solution) or 1 (a correct solution)
    def binary_fitness(self, k, problem_row_sums, problem_col_sums):
        diff = 0
        curr_row_sums = self.calc_rows(k)
        curr_col_sums = self.calc_cols(k)
        if problem_col_sums == curr_col_sums and problem_row_sums == curr_row_sums:
            self.fitness = 0
            return 0
        else:
            self.fitness = 1
            return 1
        
    # TODO: should combine calc_rows and calc_cols so I don't have to recompute curr_solution twice
    def calc_rows(self, k):
        curr_solution = [0 for _ in range(k**2)]
        for i in range(k**2):
            if self.chromosome[i] == 1:
                curr_solution[i] = self.problem[i]
        row_split = [curr_solution[x:x+k] for x in range(0, k**2, k)]
        row_sums = [sum(r) for r in row_split]
        return row_sums
    
    def calc_cols(self, k):
        curr_solution = [0 for _ in range(k**2)]
        for i in range(k**2):
            if self.chromosome[i] == 1:
                curr_solution[i] = self.problem[i]
        col_split = [[curr_solution[i] for i in range(j, k**2, k) ] for j in range(k)]
        col_sums = [sum(c) for c in col_split]
        return col_sums
    
    def pretty_print_chromosome(self):
        output = ['' for _ in range(len(self.chromosome))]
        for i in range(len(self.chromosome)):
            if self.chromosome[i] == 1:
                output[i] = str(self.problem[i])
            else:
                output[i] = 'X'
        temp = [output[x:x+self.prob_size] for x in range(0, self.prob_size**2, self.prob_size)]
        for r in temp:
            print(r)
        print("row values", self.calc_rows(self.prob_size))
        print("col values", self.calc_cols(self.prob_size))
        print('\n')
            
    

In [5]:
K = 5
POPULATION_SIZE = 30
# works best with 0.2 to 0.4 mutation rate...
MUTATION_RATE = 0.4
GENERATIONS = 200

current_problem = Sumplete(K)
print("grid and totals:")
current_problem.print_grid_and_totals()
print("\n")

print("solution grid:")
current_problem.print_sol_grid()
print("\n")

GA_solver(current_problem.get_grid_and_totals(), POPULATION_SIZE, MUTATION_RATE, GENERATIONS, binary_fitness=False)

grid and totals:
[7, 5, 1, 5, 1]
[9, 7, 3, 6, 7]
[9, 1, 3, 4, 2]
[7, 8, 6, 8, 9]
[7, 9, 1, 7, 4]


[13, 17, 4, 38, 12]
[14, 21, 14, 15, 20]


solution grid:
[7, 5, 1, -1, -1]
[-1, 7, 3, -1, 7]
[-1, 1, 3, -1, -1]
[7, 8, 6, 8, 9]
[-1, -1, 1, 7, 4]


solving problem grid:
[[7, 5, 1, 5, 1], [9, 7, 3, 6, 7], [9, 1, 3, 4, 2], [7, 8, 6, 8, 9], [7, 9, 1, 7, 4]]
Gen 0: Best individual fitness = 0.997229916897507
Gen 1: Best individual fitness = 0.9958847736625515
Gen 2: Best individual fitness = 0.9958847736625515
Gen 3: Best individual fitness = 0.9958847736625515
Gen 4: Best individual fitness = 0.9957446808510638
Gen 5: Best individual fitness = 0.9887640449438202
Gen 6: Best individual fitness = 0.9887640449438202
Gen 7: Best individual fitness = 0.9887640449438202
Gen 8: Best individual fitness = 0.9887640449438202
Gen 9: Best individual fitness = 0.9887640449438202
Gen 10: Best individual fitness = 0.9887640449438202
Gen 11: Best individual fitness = 0.9887640449438202
Gen 12: Best indivi

In [18]:
K = 5
# works with 0.4 mutation rate...
MUTATION_RATE = 0.2
GENERATIONS = 200

POPULATION_SIZE = 120
high_pop_bests = []
for i in range(10):
    ga_s = GA_solver(Sumplete(K).get_grid_and_totals(), POPULATION_SIZE, MUTATION_RATE, GENERATIONS, binary_fitness=False)
    high_pop_bests.append(ga_s.get_best())
    
POPULATION_SIZE = 30
low_pop_bests = []
for i in range(10):
    ga_s = GA_solver(Sumplete(K).get_grid_and_totals(), POPULATION_SIZE, MUTATION_RATE, GENERATIONS, binary_fitness=False)
    low_pop_bests.append(ga_s.get_best())

POPULATION_SIZE = 240
highest_pop_bests = []
for i in range(10):
    ga_s = GA_solver(Sumplete(K).get_grid_and_totals(), POPULATION_SIZE, MUTATION_RATE, GENERATIONS, binary_fitness=False)
    highest_pop_bests.append(ga_s.get_best())

K = 6
POPULATION_SIZE = 120
high_pop_bests_6 = []
for i in range(10):
    ga_s = GA_solver(Sumplete(K).get_grid_and_totals(), POPULATION_SIZE, MUTATION_RATE, GENERATIONS, binary_fitness=False)
    high_pop_bests_6.append(ga_s.get_best())
    
POPULATION_SIZE = 30
    
low_pop_bests_6 = []
for i in range(10):
    ga_s = GA_solver(Sumplete(K).get_grid_and_totals(), POPULATION_SIZE, MUTATION_RATE, GENERATIONS, binary_fitness=False)
    low_pop_bests_6.append(ga_s.get_best())

POPULATION_SIZE = 240
highest_pop_bests_6 = []
for i in range(10):
    ga_s = GA_solver(Sumplete(K).get_grid_and_totals(), POPULATION_SIZE, MUTATION_RATE, GENERATIONS, binary_fitness=False)
    highest_pop_bests_6.append(ga_s.get_best())

print("k=5:")
print(highest_pop_bests)
print(high_pop_bests)
print(low_pop_bests)
print()
print("k=6:")
print(highest_pop_bests_6)
print(high_pop_bests_6)
print(low_pop_bests_6)


solving problem grid:
[[7, 7, 1, 3, 6], [4, 1, 6, 3, 1], [7, 8, 9, 4, 1], [9, 6, 6, 1, 2], [7, 8, 3, 2, 8]]
Gen 0: Best individual fitness = 0.9960474308300395
Gen 1: Best individual fitness = 0.9948186528497409
Gen 2: Best individual fitness = 0.9896907216494846
Gen 3: Best individual fitness = 0.9714285714285714
Gen 4: Best individual fitness = 0.9714285714285714
Gen 5: Best individual fitness = 0.9655172413793104
Gen 6: Best individual fitness = 0.9655172413793104
Gen 7: Best individual fitness = 0.962962962962963
Gen 8: Best individual fitness = 0.962962962962963
Gen 9: Best individual fitness = 0.96
Gen 10: Best individual fitness = 0.9565217391304348
Gen 11: Best individual fitness = 0.9565217391304348
Gen 12: Best individual fitness = 0.9565217391304348
Gen 13: Best individual fitness = 0.8571428571428572
Gen 14: Best individual fitness = 0.8571428571428572
Gen 15: Best individual fitness = 0.8571428571428572
Gen 16: Best individual fitness = 0.8571428571428572
Gen 17: Best indi

In [21]:
# Binary fitness performance across different k values
K = 3
POPULATION_SIZE = 30
# works with 0.4 and 0.1 mutation rate, could probs be tuned a bit more
MUTATION_RATE = 0.2
GENERATIONS = 200

# correct solution? (fitness)
binary_results = {k:[] for k in range(3, 7)}
# convergence speed?
bin_conv_results = {k:[] for k in range(3, 7)}
for _ in range(10):
    for k_val in range(3,7):
        ga = GA_solver(Sumplete(k_val).get_grid_and_totals(), POPULATION_SIZE, MUTATION_RATE, GENERATIONS, True)
        binary_results[k_val].append(ga.get_best())
        bin_conv_results[k_val].append(ga.solution_foiund_in_gen)
        

print(binary_results)
print(bin_conv_results)


solving problem grid:
[[8, 1, 8], [3, 2, 9], [4, 5, 6]]
Gen 0: Best individual fitness = 1
Gen 1: Best individual fitness = 1
Gen 2: Best individual fitness = 1
Gen 3: Best individual fitness = 1
Gen 4: Best individual fitness = 1
Gen 5: Best individual fitness = 1
Gen 6: Best individual fitness = 1
Gen 7: Best individual fitness = 1
Gen 8: Best individual fitness = 1
Gen 9: Best individual fitness = 1
Gen 10: Best individual fitness = 1
Gen 11: Best individual fitness = 1
Gen 12: Best individual fitness = 1
Gen 13: Best individual fitness = 1
Gen 14: Best individual fitness = 1
Gen 15: Best individual fitness = 1
Gen 16: Best individual fitness = 1
Gen 17: Best individual fitness = 1
Gen 18: Best individual fitness = 1
SOLUTION FOUND!
In gen: 19
['X', '1', '8']
['3', '2', 'X']
['4', '5', '6']
row values [9, 5, 15]
col values [7, 8, 14]


0
solving problem grid:
[[4, 5, 5, 8], [6, 5, 2, 3], [8, 1, 1, 5], [4, 1, 4, 3]]
Gen 0: Best individual fitness = 1
Gen 1: Best individual fitness = 

In [22]:
# scalar performance across different k values
K = 3
POPULATION_SIZE = 30
# works with 0.4 and 0.1 mutation rate, could probs be tuned a bit more
MUTATION_RATE = 0.2
GENERATIONS = 200

# correct solution? (fitness)
scalar_results = {k:[] for k in range(3, 7)}
# convergence speed?
scalar_conv_results = {k:[] for k in range(3, 7)}
for _ in range(10):
    for k_val in range(3,7):
        ga = GA_solver(Sumplete(k_val).get_grid_and_totals(), POPULATION_SIZE, MUTATION_RATE, GENERATIONS, False)
        scalar_results[k_val].append(ga.get_best())
        scalar_conv_results[k_val].append(ga.solution_foiund_in_gen)
        

print(scalar_results)
print(scalar_conv_results)

solving problem grid:
[[1, 9, 9], [8, 9, 3], [1, 3, 4]]
Gen 0: Best individual fitness = 0.9924812030075187
Gen 1: Best individual fitness = 0.987012987012987
SOLUTION FOUND!
In gen: 2
['1', '9', '9']
['X', '9', 'X']
['1', 'X', 'X']
row values [19, 9, 1]
col values [2, 18, 9]


0.0
solving problem grid:
[[9, 4, 2, 4], [4, 2, 4, 9], [3, 6, 7, 1], [7, 8, 6, 2]]
Gen 0: Best individual fitness = 0.987012987012987
Gen 1: Best individual fitness = 0.9818181818181818
Gen 2: Best individual fitness = 0.9787234042553191
Gen 3: Best individual fitness = 0.9787234042553191
Gen 4: Best individual fitness = 0.9787234042553191
Gen 5: Best individual fitness = 0.9787234042553191
Gen 6: Best individual fitness = 0.9714285714285714
Gen 7: Best individual fitness = 0.9696969696969697
Gen 8: Best individual fitness = 0.9696969696969697
Gen 9: Best individual fitness = 0.96
Gen 10: Best individual fitness = 0.96
Gen 11: Best individual fitness = 0.96
Gen 12: Best individual fitness = 0.9411764705882353
Ge

In [23]:
binary_results_avged = {k:sum(binary_results[k])/10 for k in range(3, 7)}
binary_conv_results_avged = {k:sum(bin_conv_results[k])/10 for k in range(3, 7)}
scalar_results_avged = {k:sum(scalar_results[k])/10 for k in range(3, 7)}
scalar_conv_results_avged = {k:sum(scalar_conv_results[k])/10 for k in range(3, 7)}

print("binary fitness results avged over 10 runs:", binary_results_avged)
print("binary convergence speed results avged over 10 runs:",binary_conv_results_avged)
print("scalar fitness results avged over 10 runs:",scalar_results_avged)
print("scalar convergence speed avged over 10 runs:",scalar_conv_results_avged)

binary fitness results avged over 10 runs: {3: 0.0, 4: 1.0, 5: 1.0, 6: 1.0}
binary convergence speed results avged over 10 runs: {3: 35.7, 4: 200.0, 5: 200.0, 6: 200.0}
scalar fitness results avged over 10 runs: {3: 0.0, 4: 0.06666666666666668, 5: 0.4634100199317591, 6: 0.8211531943686919}
scalar convergence speed avged over 10 runs: {3: 3.4, 4: 41.1, 5: 127.6, 6: 200.0}


In [24]:
# draw graphs of results
# doing this in another notebook as I have maxed the mem of this one I think...

In [25]:
# part c
# Discuss the effect of the GA parameters on the solution.
# I could split this into a 2 sections: 1. NFL and general param effect, 2. Effect of changing params in Sumplete context

# 1. General param effects
# i used:
# POPULATION_SIZE:
    # larger: more parallisation, quicker convergence, more likely to start in good positions
    # too large: makes algorithm take a long time to compute
    # too small: less likely to start in good positions, takes longer to converge
    
# MUTATION_RATE:
    # Too high: you risk the individuals "jumping" over a solution they were close to.
    # high: more search space exploration, by preventing premature convergence
    # low: faster convergence rate (possibly on local minima though)
    # Too low: they're all going to get stuck in local minima.

# GENERATIONS:
    # Too large: you're going to have a long epoch time, restricting how many chances each individual has to explore it's neighbourhood.
    # Too small: you don't get good coverage of the search space.

# other params I could've used and should discuss
# XO_RATE
    # high: faster convergence (but on possible non-optimal solutions), more crossover == more diversity, more exploitation 
    # low: lets more individuals move to the next generation unchanged, loss of diversity
    
# KILL_PERCENTAGE, how many Individuals should I kill off? My GA is effectively 50% 
    # Too high: could kill off solution leading to global minima
    # Too low: slow convergence speed as we are letting the worst solutions breed with our best solutions, making mid children


# 2. In a Sumplete context
# discuss experiment results in report


# references?
# https://www.sciencedirect.com/science/article/pii/S0304397502000944: practically the NFL theorem does not hold?
# choosing xo and mutation params: https://www.researchgate.net/publication/337868866_Choosing_Mutation_and_Crossover_Ratios_for_Genetic_Algorithms-A_Review_with_a_New_Dynamic_Approach

In [26]:
# part c experiments on K=4, using scaling fitness
import numpy as np
K = 4
POPULATION_SIZE = 30
MUTATION_RATE = 0.4
GENERATIONS = 200

# Effect of mutation rate:
part_c_fitnesses = {m_r:[] for m_r in np.arange(0.0, 1.1, 0.1)}
part_c_convergences = {m_r:[] for m_r in np.arange(0.0, 1.1, 0.1)}
for _ in range(5):
    for mr in np.arange(0.0, 1.1, 0.1):
        MUTATION_RATE = mr
        ga = GA_solver(Sumplete(K).get_grid_and_totals(), POPULATION_SIZE, MUTATION_RATE, GENERATIONS, False)
        part_c_fitnesses[mr].append(ga.get_best())
        part_c_convergences[mr].append(ga.solution_foiund_in_gen)

part_c_fitnesses_avg = {m_r:sum(part_c_fitnesses[m_r])/5 for m_r in np.arange(0.0,1.1,0.1)}
part_c_convergence_avg = {m_r:sum(part_c_convergences[m_r])/5 for m_r in np.arange(0.0,1.1,0.1)}


# Effect of Population Size:
MUTATION_RATE = 0.4
pop_size_fitnesses = {p_z:[] for p_z in [1,5,10,15,20,25,30,35,40,45,50,60,70,80,90,100]}
pop_size_convs = {p_z:[] for p_z in [1,5,10,15,20,25,30,35,40,45,50,60,70,80,90,100]}
for _ in range(5):
    for pz in [1,5,10,15,20,25,30,35,40,45,50,60,70,80,90,100]:
        POPULATION_SIZE = pz
        ga = GA_solver(Sumplete(K).get_grid_and_totals(), POPULATION_SIZE, MUTATION_RATE, GENERATIONS, False)
        pop_size_fitnesses[pz].append(ga.get_best())
        pop_size_convs[pz].append(ga.solution_foiund_in_gen)

print(pop_size_fitnesses)
print(pop_size_convs)
pz_fitness_avg = {pz:sum(pop_size_fitnesses[pz])/5 for pz in [1,5,10,15,20,25,30,35,40,45,50,60,70,80,90,100]}
pz_convergence_avg = {pz:sum(pop_size_convs[pz])/5 for pz in [1,5,10,15,20,25,30,35,40,45,50,60,70,80,90,100]}


# Effect of # of Generations
POPULATION_SIZE=30
gen_size_fitnesses = {gz:[] for gz in range(0, 501, 100)}
gen_size_convs = {gz:[] for gz in range(0, 501, 100)}
for _ in range(5):
    for gz in range(0, 501, 100):
        GENERATIONS = gz
        ga = GA_solver(Sumplete(K).get_grid_and_totals(), POPULATION_SIZE, MUTATION_RATE, GENERATIONS, False)
        gen_size_fitnesses[gz].append(ga.get_best())
        gen_size_convs[gz].append(ga.solution_foiund_in_gen)

print(gen_size_fitnesses)
print(gen_size_convs)
gen_fitness_avg = {gz:sum(gen_size_fitnesses[gz])/5 for gz in range(0, 501, 100)}
gen_convergence_avg = {gz:sum(gen_size_convs[gz])/5 for gz in range(0, 501, 100)}


solving problem grid:
[[4, 6, 3, 9], [8, 6, 4, 4], [1, 3, 2, 4], [6, 5, 6, 5]]
Gen 0: Best individual fitness = 0.9937106918238994
Gen 1: Best individual fitness = 0.9714285714285714
Gen 2: Best individual fitness = 0.9714285714285714
Gen 3: Best individual fitness = 0.9714285714285714
Gen 4: Best individual fitness = 0.9714285714285714
Gen 5: Best individual fitness = 0.9714285714285714
Gen 6: Best individual fitness = 0.9714285714285714
Gen 7: Best individual fitness = 0.9714285714285714
Gen 8: Best individual fitness = 0.9714285714285714
Gen 9: Best individual fitness = 0.9714285714285714
Gen 10: Best individual fitness = 0.9714285714285714
Gen 11: Best individual fitness = 0.9714285714285714
Gen 12: Best individual fitness = 0.9714285714285714
Gen 13: Best individual fitness = 0.9714285714285714
Gen 14: Best individual fitness = 0.9714285714285714
Gen 15: Best individual fitness = 0.9714285714285714
Gen 16: Best individual fitness = 0.9714285714285714
Gen 17: Best individual fitnes

In [28]:
print("mutation rates: fitness & convergence")
#print("mr fitness:", part_c_fitnesses)
print(part_c_fitnesses_avg)
print("mr convergence:", part_c_convergence_avg)
#print(part_c_convergence_avg)

print('\n')
print("pop_size fitness: (note: is not a very helpful stat)")
print(pz_fitness_avg)
print("pop_size avg convergence generation:")
print(pz_convergence_avg)

print('\n')
print("gen_size fitness:")
print(gen_fitness_avg)
print("gen_size avg convergence generation !NOTE! THIS DOESNT MAKE SENSE TO USE:")
print(gen_convergence_avg)
print("note 200 could look could here bc that was what was used in testing, so some tuning done for this value in partiulcar")

mutation rates: fitness & convergence
{0.0: 0.7752679504403643, 0.1: 0.3619047619047619, 0.2: 0.0, 0.30000000000000004: 0.0, 0.4: 0.0, 0.5: 0.0, 0.6000000000000001: 0.13333333333333336, 0.7000000000000001: 0.7609022556390979, 0.8: 0.6795959595959596, 0.9: 0.8346951294319715, 1.0: 0.9421534986752379}
mr convergence: {0.0: 161.2, 0.1: 84.2, 0.2: 31.8, 0.30000000000000004: 19.8, 0.4: 64.0, 0.5: 83.2, 0.6000000000000001: 135.8, 0.7000000000000001: 200.0, 0.8: 170.8, 0.9: 200.0, 1.0: 200.0}


pop_size fitness:
{1: 0.18947368421052632, 5: 0.13333333333333336, 10: 0.0, 15: 0.0, 20: 0.18181818181818182, 25: 0.0, 30: 0.13333333333333336, 35: 0.0, 40: 0.0, 45: 0.0, 50: 0.0, 60: 0.0, 70: 0.0, 80: 0.0, 90: 0.17142857142857143, 100: 0.0}
pop_size avg convergence generation:
{1: 134.6, 5: 100.8, 10: 69.4, 15: 72.4, 20: 70.4, 25: 39.8, 30: 77.6, 35: 25.4, 40: 58.0, 45: 42.2, 50: 32.0, 60: 34.6, 70: 48.8, 80: 29.0, 90: 51.4, 100: 15.6}


gen_size fitness:
{0: 0.9968181398493059, 100: 0.177777777777777